In [2]:
# 1. Library install
!pip install librosa
!pip install graphviz
!pip install tqdm

!apt-get install graphviz

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
'apt-get' is not recognized as an internal or external command,
operable program or batch file.


In [8]:
# 2. import library
"""'''
os,glob : dir와 파일들에 접근하기 위한 Lib
tensorflow, keras : 학습 및 모델 생성을 위한 Lib
numpy : Matrix transpose 등 연산을 휘한 Lib
librosa : feature extraction 및 음성 파일을 처리하기 위한 lib
tqdm : iteration progress bar 생성 lib
sklearn :scikit learn, machine learning을 지원해주는 lib
'''"""

import os
import tensorflow as tf
import keras
import numpy as np
import librosa
import librosa.display
import glob
from tqdm.notebook import tqdm
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import np_utils
from keras.utils.vis_utils import model_to_dot
%matplotlib inline 
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

print( "Python packages are imported." )


Python packages are imported.


In [33]:
# MFCC 추출 및 데이터 전처리(sklearn.preprocessing.scale)
import sklearn.preprocessing 
from PIL import Image

def mfcc_extract(filename):
    y,sr  = librosa.load(filename,sr = 16000)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=100,n_fft=int(0.025*sr),hop_length=int(0.01*sr))

    mfcc = sklearn.preprocessing.scale(mfcc, axis=1)
    pad2d = lambda a, i: a[:, 0:i] if a.shape[1] > i else np.hstack((a, np.zeros((a.shape[0], i-a.shape[1]))))
    padded_mfcc = pad2d(mfcc, 90)
    #librosa.display.specshow(padded_mfcc, sr=16000, x_axis='time')  #시각화하기

    #delta=librosa.feature.delta(mfcc)
    #delta2=librosa.feature.delta(mfcc,order=2)
    #con_mfcc=np.concatenate((mfcc,delta,delta2),axis=0)
    #print(y)
    #print(len(y))
    #print('Sampling rate (Hz): %d' %sr)
    #print('Audio length (seconds): %.2f' % (len(y) / sr)) #음악의 길이(초) = 음파의 길이/Sampling rate
    return padded_mfcc
pass # -- mfcc_extract
  
# parse_audio_files는 parent_dir, sub_dirs를 변수로 받음
# parent_dir 아래의 sub_dirs안에서 wav파일들을 읽고 mfcc feature를 추출 
def parse_audio_files( data_dir ):
    # 여기서는 MFCC 만 추출함, 레이블 데이터는 실제 학습시에 자동으로 부여함.
    features = []
    sub_dirs = os.listdir( data_dir )
    
    print( f"Parsing auto files from {data_dir} ...." )
    
    img_file_cnt = 0 
    
    for sub_dir in sub_dirs :
        # 그냥 폴더 순서 대로 레이블을 지정함.
        print( "sub_dir = ", sub_dir  )        
        
        for fn in glob.glob(os.path.join( data_dir, sub_dir, "*.wav")):
            #print mfcc_extract(fn).shape
            features.append(mfcc_extract(fn)) 

            # 같은 폴더에 저장함.
            # img, wav 폴더를 구분하지 않음.
            fn_new = fn.replace('.wav','.png') 
            
            plt.imsave( fn_new,  mfcc_extract(fn))
            
            img_file_cnt += 1
        pass
    pass

    print( f"{img_file_cnt} image files converted from audio file are saved." )

    return features 
pass # -- parse_audio_files

print( "Python functions are defined." )

# File들로부터 feature를 추출
# 모든 음성파일들은 1초이며 sampling rate는 160kHz, bit는 16bit이다. (확인필요!!! - https://melon1024.github.io/ssc/)
# 따라서 mfcc feature들은 한 파일당 13*501의 형태를 가지게 된다.

# train_cat의 내용을 확인
# 학습 폴더 

# Warning 메시지가 너무 많아서, 잠시 경고 메시지를 숨김
# 데이터가 제대로 처리되지 못할 가능성도 있습니다.
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='sklearn')

# 학습 /테스트 데이터 MFCC 추출
# 학습 폴더와 테스트 폴더의 MFCC 를 추출하여 png 이미지 파일도 저장한다.
data_dirs = [ './data/train', './data/test' ] 
for data_dir in data_dirs : 
    features = parse_audio_files( data_dir ) 

    # features 확인 
    # features는 python기본 타입 중 하나인 list이며 (13*501) ndarray를 원소로 가지고 있음

    print( "feature len = ", len(features), "feature[0] shape", features[0].shape)
    print( "type(feature) = ", type(features), "type(feature[0]) = ", type(features[0]))
    print()
pass

print( "Extracting MFCC is finished.")

Python functions are defined.
Parsing auto files from ./data/train ....
sub_dir =  0-go
sub_dir =  1-left
sub_dir =  2-right
sub_dir =  3-stop
401 image files converted from audio file are saved.
feature len =  401 feature[0] shape (100, 90)
type(feature) =  <class 'list'> type(feature[0]) =  <class 'numpy.ndarray'>

Parsing auto files from ./data/test ....
sub_dir =  0-go
sub_dir =  1-left
sub_dir =  2-right
sub_dir =  3-stop
80 image files converted from audio file are saved.
feature len =  80 feature[0] shape (100, 90)
type(feature) =  <class 'list'> type(feature[0]) =  <class 'numpy.ndarray'>

Extracting MFCC is finished.


In [34]:
# Part 1 - Building the CNN 
# Importing the Keras libraries and packages

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (100, 90, 3), activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Part 2 - Fitting the CNN to the images
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory( './data/train',
                                                 target_size = (100, 90),
                                                 batch_size = 10,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory( './data/test',
                                            target_size = (100, 90),
                                            batch_size = 10,
                                            class_mode = 'binary')

# 데이터를 무작위로 섞는다.???? 넣으면 오류나나
#import random
#random.shuffle(training_set)
#random.shuffle(test_set)

classifier.fit(training_set,
                steps_per_epoch = 30,
                epochs = 50,
                validation_data = test_set,
                validation_steps = 3)

output = classifier.predict(test_set, steps=5)
print(test_set.class_indices)
print(output)

Found 401 images belonging to 4 classes.
Found 80 images belonging to 4 classes.
Epoch 1/50
30/30 [==============================] - 4s 114ms/step - loss: -448.8201 - accuracy: 0.2268 - val_loss: -2144.2239 - val_accuracy: 0.2000
Epoch 2/50
30/30 [==============================] - 2s 76ms/step - loss: -9784.3447 - accuracy: 0.2612 - val_loss: -37556.4570 - val_accuracy: 0.3000
Epoch 3/50
30/30 [==============================] - 2s 70ms/step - loss: -83943.4766 - accuracy: 0.2612 - val_loss: -186947.8281 - val_accuracy: 0.1667
Epoch 4/50
30/30 [==============================] - 2s 69ms/step - loss: -354236.6875 - accuracy: 0.2200 - val_loss: -989847.4375 - val_accuracy: 0.3000
Epoch 5/50
30/30 [==============================] - 2s 68ms/step - loss: -1091693.1250 - accuracy: 0.2646 - val_loss: -1156867.5000 - val_accuracy: 0.2000
Epoch 6/50
30/30 [==============================] - 2s 67ms/step - loss: -3420149.7500 - accuracy: 0.2509 - val_loss: -3418534.5000 - val_accuracy: 0.1667
Epoch